In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from math import log2
from IPython.display import display
%matplotlib inline

from sklearn.metrics import r2_score
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import median_absolute_error
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.pipeline import make_pipeline
import time
import matplotlib as mpl

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor



from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,QuantileTransformer,normalize
from sklearn.model_selection import train_test_split, learning_curve, ShuffleSplit
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split

import pathlib
import os
import sys

# for selection the right path
import os,sys,inspect,pathlib
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from common.model_trainer_reg import *
from common.regression_plotfunctions import *

from GD.LinearRegression import LinearRegression
from KNN.KNNRegressor import KNNRegressor

import moneyball_preprocession


In [2]:
# Custom functions

module_path = pathlib.Path(os.getcwd()).parent
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
print(sys.path)

from common.dataset_grabber import get_data_path

['/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2/Moneyball', '', '/home/alexander/ngsuite/ngsolve-install/lib/python3/dist-packages', '/home/alexander/ngsuite/ngsolve-install/bin', '/home/alexander/.local/bin', '/usr/local/sbin', '/usr/local/bin', '/usr/sbin', '/usr/bin', '/sbin', '/bin', '/usr/games', '/usr/local/games', '/snap/bin', '/home/alexander/.vscode/extensions/ms-toolsai.jupyter-2020.11.399280825/pythonFiles', '/home/alexander/.vscode/extensions/ms-toolsai.jupyter-2020.11.399280825/pythonFiles/lib/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/home/alexander/.local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages

In [3]:
datapath = get_data_path("Moneyball", "baseball.csv")
display(datapath)

PosixPath('/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2/Datasets/Moneyball/baseball.csv')

In [4]:
df_raw = pd.read_csv(datapath)

# Train and Test

In [5]:
X, Y = moneyball_preprocession.preprocessing(df_raw, drop_missing_values=False)

## SGD-Regression

In [7]:
MODEL = SGDRegressor
NAME = "SGD"
PATH = "out/"+NAME+"/"
params = {"alpha" : [0.001,0.001],
          "max_iter" : [1000,10000]}

n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Varerror=mean_gamma_deviance, Error="MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_SGD.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_SGD.csv")
display(results)




,alpha,max_iter,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,0.001,1000,0.011837,0.000388,0.876465,15.992029,4,1232,616,616,0.002715
1,0.001,10000,0.006242,0.000304,0.876347,16.007396,4,1232,616,616,0.002717
2,0.001,1000,0.005023,0.000297,0.876475,15.990819,4,1232,616,616,0.002712
3,0.001,10000,0.005316,0.000252,0.876578,15.977408,4,1232,616,616,0.002707


,alpha,max_iter,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,0.001,1000,0.007432,0.000301,1,0.876544,15.981898,4,1232,616,616,0.002707
1,0.001,10000,0.009508,0.000306,1,0.876468,15.991677,4,1232,616,616,0.002709
2,0.001,1000,0.005795,0.000306,1,0.876676,15.964716,4,1232,616,616,0.002709
3,0.001,10000,0.004878,0.000215,1,0.876424,15.997400,4,1232,616,616,0.002713
4,0.001,1000,0.005116,0.000249,2,0.869490,15.415401,4,1232,616,616,0.002498
5,0.001,10000,0.003241,0.000220,2,0.868845,15.491483,4,1232,616,616,0.002510
6,0.001,1000,0.003470,0.000195,2,0.868877,15.487772,4,1232,616,616,0.002511
7,0.001,10000,0.003068,0.000197,2,0.869096,15.461869,4,1232,616,616,0.002500


## My SGD-Regression

In [8]:
MODEL = LinearRegression
params = {"alpha" : [0.001,0.001],
          "max_iter" : [1000,10000,100000]}
n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Varerror=mean_gamma_deviance, Error="MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "my_TTS_SGD.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "my_CV_SGD.csv")
display(results)

,alpha,max_iter,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,0.001,1000,0.044135,0.000031,0.875578,16.106947,4,1232,616,616,0.002735
1,0.001,10000,0.380690,0.000044,0.875562,16.108963,4,1232,616,616,0.002736
2,0.001,100000,3.516646,0.000030,0.875562,16.108963,4,1232,616,616,0.002736
3,0.001,1000,0.036451,0.000028,0.875578,16.106947,4,1232,616,616,0.002735
4,0.001,10000,0.328059,0.000028,0.875562,16.108963,4,1232,616,616,0.002736
5,0.001,100000,3.269195,0.000029,0.875562,16.108963,4,1232,616,616,0.002736


,alpha,max_iter,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,0.001,1000,0.041229,0.000041,1,0.875578,16.106947,4,1232,616,616,0.002735
1,0.001,10000,0.325387,0.000038,1,0.875562,16.108963,4,1232,616,616,0.002736
2,0.001,100000,3.300977,0.000029,1,0.875562,16.108963,4,1232,616,616,0.002736
3,0.001,1000,0.033530,0.000028,1,0.875578,16.106947,4,1232,616,616,0.002735
4,0.001,10000,0.328313,0.000031,1,0.875562,16.108963,4,1232,616,616,0.002736
5,0.001,100000,3.402775,0.000030,1,0.875562,16.108963,4,1232,616,616,0.002736
6,0.001,1000,0.037652,0.000028,2,0.870292,15.320653,4,1232,616,616,0.002480
7,0.001,10000,0.320341,0.000028,2,0.870294,15.320330,4,1232,616,616,0.002480
8,0.001,100000,3.448231,0.000029,2,0.870294,15.320330,4,1232,616,616,0.002480
9,0.001,1000,0.031500,0.000027,2,0.870292,15.320653,4,1232,616,616,0.002480


## KNN-Regression

In [9]:
MODEL = KNeighborsRegressor
params = {"n_neighbors": [5],
        "weights" : ["uniform"],
        "algorithm": ["brute","kd_tree","ball_tree"]}
NAME = "KNN"
PATH = "out/"+NAME+"/"

n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Varerror=mean_gamma_deviance, Error="MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_KNN.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_KNN.csv")
display(results)

,n_neighbors,weights,algorithm,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,5,uniform,brute,0.002914,0.031884,0.785048,27.826299,4,1232,616,616,0.004589
1,5,uniform,kd_tree,0.001717,0.016688,0.785048,27.826299,4,1232,616,616,0.004589
2,5,uniform,ball_tree,0.002003,0.014089,0.785048,27.826299,4,1232,616,616,0.004589


,n_neighbors,weights,algorithm,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,5,uniform,brute,0.001447,0.020011,1,0.785048,27.826299,4,1232,616,616,0.004589
1,5,uniform,kd_tree,0.002884,0.035835,1,0.785048,27.826299,4,1232,616,616,0.004589
2,5,uniform,ball_tree,0.012427,0.016666,1,0.785048,27.826299,4,1232,616,616,0.004589
3,5,uniform,brute,0.000473,0.016200,2,0.775452,26.522792,4,1232,616,616,0.004234
4,5,uniform,kd_tree,0.001881,0.015910,2,0.775452,26.522792,4,1232,616,616,0.004234
5,5,uniform,ball_tree,0.001755,0.014412,2,0.775452,26.522792,4,1232,616,616,0.004234


## my KNN-Regression

In [10]:
MODEL = KNNRegressor
params = {"n_neighbors" : [5],
           "p": [2]}
           #"weights": ["uniform"]}
NAME = "KNN"
PATH = "out/"+NAME+"/"

n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Varerror=mean_gamma_deviance, Error="MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "my_TTS_KNN.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "my_CV_KNN.csv")
display(results)

chunking...


,n_neighbors,p,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,5,2,0.000011,0.060127,0.785048,27.826299,4,1232,616,616,0.004589


chunking...
chunking...


,n_neighbors,p,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,5,2,0.000011,0.061449,1,0.785048,27.826299,4,1232,616,616,0.004589
1,5,2,0.000010,0.055298,2,0.775452,26.522792,4,1232,616,616,0.004234


## RF-Regression

In [11]:
MODEL = RandomForestRegressor
params = {"n_estimators" : [100]}
NAME = "RF"
PATH = "out/"+NAME+"/"


n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Varerror=mean_gamma_deviance, Error="MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_RF.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_RF.csv")
display(results)

,n_estimators,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,100,0.359728,0.020802,0.851053,19.281718,4,1232,616,616,0.00318


,n_estimators,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,100,0.416769,0.017546,1,0.847606,19.728006,4,1232,616,616,0.003242
1,100,0.338540,0.017324,2,0.838903,19.028135,4,1232,616,616,0.003004


## DT-Regression

In [12]:
MODEL = DecisionTreeRegressor
params = {"criterion": ["mse"]}
NAME = "DT"
PATH = "out/"+NAME+"/"


n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Varerror=mean_gamma_deviance, Error="MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_DT.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_DT.csv")
display(results)

,criterion,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,mse,0.005036,0.000461,0.687824,40.412338,4,1232,616,616,0.006535


,criterion,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,mse,0.004497,0.000527,1,0.690646,40.047078,4,1232,616,616,0.006559
1,mse,0.005310,0.000471,2,0.675149,38.370130,4,1232,616,616,0.006032
